In [5]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
%matplotlib inline

In [6]:
start = datetime.datetime(2022,4,1)
end = datetime.datetime(2022,4,25)

In [7]:
share = data.DataReader('^NSEBANK', 'yahoo', start, end)

In [8]:
returns = pd.DataFrame()

In [9]:
returns['High'] =share['High'].pct_change()*100
returns['Low'] =share['Low'].pct_change()*100
returns['Open'] =share['Open'].pct_change()*100
returns['Close'] =share['Close'].pct_change()*100
returns['Volume'] =share['Volume'].pct_change()*100

In [10]:
returns.tail()

,High,Low,Open,Close,Volume
Date,,,,,
2022-04-19,0.426608,-1.413369,-0.071532,-1.054748,18.261345
2022-04-20,-1.642352,0.522598,-0.880813,-0.073478,-29.079982
2022-04-21,1.094105,0.779886,0.083601,1.380158,-13.494133
2022-04-22,-0.905912,-1.110569,0.002867,-2.095147,5.576488
2022-04-25,-0.844204,-1.333930,-1.976759,0.104319,-100.000000


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Optional Plotly Method Imports
import plotly
import cufflinks as cf
cf.go_offline()
returns[['Open', 'High', 'Low', 'Close']].iplot(kind='bar')

In [12]:
share['Volume'].iplot(kind='bar')

In [13]:
share[['Open', 'High', 'Low','Close']].iplot('candle')

In [14]:
returns['Volume'].iplot(kind='bar')

In [15]:
nifty=pd.read_excel('INDEX.xlsx','BANKNIFTY')
nifty.set_index('Ticker',inplace=True)
nifty.head()

,Weightage
Ticker,
AXISBANK.NS,10.46
AUBANK.NS,1.16
BANDHANBNK.NS,1.08
FEDERALBNK.NS,0.91
HDFCBANK.NS,32.70


In [16]:
tracking_stocks=pd.DataFrame()
tracking_stocks.index=nifty.index
positive=0
negative=0
neutral=0
tracking_stocks['Change']=0
tracking_stocks['Volume']=0
tracking_stocks['Value']=0
convert_dict = {'Change':float,
                'Volume': float,
                'Value':float
               }
tracking_stocks=tracking_stocks.astype(convert_dict)
nifty_50=pd.DataFrame()
range_date = pd.date_range(start,end)
final_nifty=pd.DataFrame()
final_nifty=range_date.to_frame().drop(columns=0,axis=1)
final_nifty['Relative Value']=0
final_nifty['Total Volume']=0
final_nifty['Sigma Value']=0
convert_dict = {'Relative Value':float,
                'Total Volume': float,
                'Sigma Value':float
               }
final_nifty=final_nifty.astype(convert_dict)
final_nifty.head()

,Relative Value,Total Volume,Sigma Value
2022-04-01,0.0,0.0,0.0
2022-04-02,0.0,0.0,0.0
2022-04-03,0.0,0.0,0.0
2022-04-04,0.0,0.0,0.0
2022-04-05,0.0,0.0,0.0


In [17]:
for stock in list(nifty.index):
    nifty_share=pd.DataFrame()
    nifty_share = data.DataReader(stock, 'yahoo', start, end)    
    nifty_share['Change']=((nifty_share['Close']-nifty_share['Open'])/nifty_share['Open'])*100
    final_nifty = final_nifty.merge(nifty_share,left_index=True, right_index=True,how='left')
    final_nifty['Change'].fillna(0,inplace = True)
    final_nifty['Volume'].fillna(0,inplace=True)
    final_nifty['Sigma Value'].fillna(0,inplace=True)
    final_nifty['Change']=final_nifty['Change']*float(nifty.loc[stock])
    final_nifty['Value']=final_nifty['Change']*final_nifty['Volume']
    final_nifty['Sigma Value']= final_nifty['Sigma Value']+final_nifty['Value']
    final_nifty['Relative Value']=final_nifty['Relative Value']+final_nifty['Change']
    final_nifty['Total Volume']=final_nifty['Total Volume']+final_nifty['Volume']
    tracking_stocks.loc[stock]['Change']=(nifty_share.loc[end]['Change'])
    tracking_stocks.loc[stock]['Volume']=nifty_share.loc[end]['Volume']
    tracking_stocks.loc[stock]['Value']=(nifty_share.loc[end]['Change']*nifty_share.loc[end]['Volume'])/1000
    if tracking_stocks.loc[stock]['Change']>0:
        positive=positive+1
    elif tracking_stocks.loc[stock]['Change']==0:
        neutral=neutral+1
    else:
        negative=negative+1
    final_nifty.drop(columns=['Open','High','Low','Close','Volume','Adj Close','Change','Value'],axis=1,inplace=True)


In [18]:
final_nifty['Total']=final_nifty['Relative Value']+final_nifty['Total Volume']
final_nifty=final_nifty[final_nifty['Total']>0]
final_nifty['Sigma Value']=final_nifty['Sigma Value']/100000
final_nifty.drop(columns=['Total'],axis=1,inplace=True)

In [19]:
print('Positive',positive)
print('Negative',negative)
print('Neutral',neutral)

Positive 8
Negative 4
Neutral 0


In [20]:
tracking_stocks.sort_values(by=['Change'], ascending=False)

,Change,Volume,Value
Ticker,,,
BANDHANBNK.NS,1.914242,10687696.0,20458.836141
KOTAKBANK.NS,1.547723,2149135.0,3326.264598
HDFCBANK.NS,1.166667,10462911.0,12206.729500
AXISBANK.NS,1.126945,8022336.0,9040.728126
ICICIBANK.NS,0.608572,35414004.0,21551.966915
FEDERALBNK.NS,0.421054,12388786.0,5216.350846
AUBANK.NS,0.289374,1039236.0,300.727579
INDUSINDBK.NS,0.164020,2096903.0,343.933762
SBIN.NS,-0.050505,14054165.0,-709.806313


In [21]:
final_nifty['Relative Value'].iplot(kind='bar')

In [22]:
final_nifty['Total Volume'].iplot(kind='bar')

In [23]:
final_nifty['Sigma Value'].iplot(kind='bar')

In [24]:
final_nifty

,Relative Value,Total Volume,Sigma Value
2022-04-01,212.489012,231012418.0,24471.164412
2022-04-04,231.665516,293841195.0,85438.942080
2022-04-05,-171.855126,201013119.0,-23679.324098
2022-04-06,-34.786202,172713429.0,-3696.648338
2022-04-07,29.090076,215500132.0,-605.360565
2022-04-08,28.910877,224327883.0,2843.278910
2022-04-11,9.503384,131529129.0,1480.874442
2022-04-12,95.851218,157510164.0,8398.237286
2022-04-13,-118.628097,131186987.0,-13354.601262
2022-04-18,-29.447014,148261283.0,-6518.363670


In [25]:
tracking_stocks=pd.DataFrame()
tracking_stocks.index=nifty.index
positive=0
negative=0
neutral=0
tracking_stocks['Change']=0
tracking_stocks['Volume']=0
tracking_stocks['Value']=0
convert_dict = {'Change':float,
                'Volume': float,
                'Value':float
               }
tracking_stocks=tracking_stocks.astype(convert_dict)
nifty_50=pd.DataFrame()
range_date = pd.date_range(start,end)
final_nifty=pd.DataFrame()
final_nifty=range_date.to_frame().drop(columns=0,axis=1)
final_nifty['Relative Value']=0
final_nifty['Total Volume']=0
final_nifty['Sigma Value']=0
convert_dict = {'Relative Value':float,
                'Total Volume': float,
                'Sigma Value':float
               }
final_nifty=final_nifty.astype(convert_dict)
final_nifty.head()

,Relative Value,Total Volume,Sigma Value
2022-04-01,0.0,0.0,0.0
2022-04-02,0.0,0.0,0.0
2022-04-03,0.0,0.0,0.0
2022-04-04,0.0,0.0,0.0
2022-04-05,0.0,0.0,0.0


In [26]:
for stock in list(nifty.index):
    nifty_share=pd.DataFrame()
    nifty_share = data.DataReader(stock, 'yahoo', start, end)    
    nifty_share['Change']=(nifty_share['Close'].pct_change())*100
    final_nifty = final_nifty.merge(nifty_share,left_index=True, right_index=True,how='left')
    final_nifty['Change'].fillna(0,inplace = True)
    final_nifty['Volume'].fillna(0,inplace=True)
    final_nifty['Sigma Value'].fillna(0,inplace=True)
    final_nifty['Change']=final_nifty['Change']*float(nifty.loc[stock])
    final_nifty['Value']=final_nifty['Change']*final_nifty['Volume']
    final_nifty['Sigma Value']= final_nifty['Sigma Value']+final_nifty['Value']
    final_nifty['Relative Value']=final_nifty['Relative Value']+final_nifty['Change']
    final_nifty['Total Volume']=final_nifty['Total Volume']+final_nifty['Volume']
    tracking_stocks.loc[stock]['Change']=(nifty_share.loc[end]['Change'])
    tracking_stocks.loc[stock]['Volume']=nifty_share.loc[end]['Volume']
    tracking_stocks.loc[stock]['Value']=(nifty_share.loc[end]['Change']*nifty_share.loc[end]['Volume'])/1000
    if tracking_stocks.loc[stock]['Change']>0:
        positive=positive+1
    elif tracking_stocks.loc[stock]['Change']==0:
        neutral=neutral+1
    else:
        negative=negative+1
    final_nifty.drop(columns=['Open','High','Low','Close','Volume','Adj Close','Change','Value'],axis=1,inplace=True)


In [27]:
final_nifty['Total']=final_nifty['Relative Value']+final_nifty['Total Volume']
final_nifty=final_nifty[final_nifty['Total']>0]
final_nifty['Sigma Value']=final_nifty['Sigma Value']/100000
final_nifty.drop(columns=['Total'],axis=1,inplace=True)

In [28]:
print('Positive',positive)
print('Negative',negative)
print('Neutral',neutral)

Positive 5
Negative 7
Neutral 0


In [29]:
tracking_stocks.sort_values(by=['Change'], ascending=True)

,Change,Volume,Value
Ticker,,,
RBLBANK.NS,-2.941178,14167734.0,-41669.823815
IDFCFIRSTB.NS,-1.872659,23337562.0,-43703.300458
PNB.NS,-1.412429,38802761.0,-54806.157242
INDUSINDBK.NS,-1.385632,2096903.0,-2905.535762
SBIN.NS,-1.168599,14054165.0,-16423.681588
FEDERALBNK.NS,-0.365534,12388786.0,-4528.518221
AUBANK.NS,-0.200481,1039236.0,-208.347241
AXISBANK.NS,0.012815,8022336.0,102.809046
KOTAKBANK.NS,0.063665,2149135.0,136.825121


In [30]:
final_nifty

,Relative Value,Total Volume,Sigma Value
2022-04-01,0.000000,231012418.0,0.000000
2022-04-04,442.415725,293841195.0,173958.505687
2022-04-05,-152.815772,201013119.0,-20334.156258
2022-04-06,-136.088254,172713429.0,-15794.132724
2022-04-07,-29.060076,215500132.0,-9308.981965
2022-04-08,43.136933,224327883.0,3297.555033
2022-04-11,-35.643203,131529129.0,-2584.117469
2022-04-12,37.079192,157510164.0,1892.256091
2022-04-13,-85.561595,131186987.0,-9713.620341
2022-04-18,-208.153065,148261283.0,-40633.072226


In [31]:
final_nifty['Relative Value'].iplot(kind='bar')

In [32]:
final_nifty['Total Volume'].iplot(kind='bar')

In [33]:
final_nifty['Sigma Value'].iplot(kind='bar')